# find regions where trades are within a certain band

In [1]:
%%time
import numpy as np
import pandas as pd
#import mplfinance as mpf
import os
from glob import glob
import matplotlib.pyplot as plt
%matplotlib inline
from collections import Counter
import talib as ta # conda activate cryt310 
tickerpair="ETCUSDT" #GBPUSDT #LTCUSDT #ETHUSDT #BTCUSDT
tickerpair="BNBUSDT" #BNBUSDT #AUDUSDT #XMRUSDT
interval="5m"
intervals=['1m','3m','5m','15m','30m','1h','2h','3h','4h','6h','8h','12h','1d','3d','1w','1Month']
percentile_list=[94,96,98]
intervals=['1m','3m','5m','15m','30m',"1h"]
tickerpairs=["ETCUSDT","GBPUSDT","LTCUSDT","ETHUSDT","BTCUSDT","BNBUSDT","AUDUSDT","XMRUSDT"]
doing=0
for tickerpair in tickerpairs:
    for interval in intervals:
        years=sorted(set([g.split("_")[2] for g in glob(f"kline_data\\{tickerpair}/*")]))
        df = [pd.read_csv(g) for g in sorted(glob(f"kline_data\\{tickerpair}/*")) if f"_{interval}.csv" in g]
        df = pd.concat(df, ignore_index=True).drop_duplicates().reset_index(drop=True)

        dfmpl=df[["open_time","open","high","low","close","volume"]]
        dfmpl = dfmpl.rename(columns={"open_time":"Date","open":"Open","high":"High","low":"Low","close":"Close","volume":"Volume"})
        dfmpl=dfmpl.set_index("Date")
        dfmpl.index = pd.to_datetime(dfmpl.index,unit="ms")
        fulllen=len(dfmpl)
        dfmpl=dfmpl.iloc[-15000:]

        xvals=np.arange(len(dfmpl.Close))
        rolling_high_std_mean = dfmpl.High.rolling(14).std()/dfmpl.High.rolling(14).mean()
        rolling_low_std_mean = dfmpl.Low.rolling(14).std()/dfmpl.Low.rolling(14).mean()
        rolling_std_mean=rolling_low_std_mean*0.5+rolling_high_std_mean*0.5
        r_std_mean_diff = rolling_std_mean.diff() 

        for percentile in percentile_list:
            thres_diff = np.percentile(r_std_mean_diff.values[np.where(~np.isnan(r_std_mean_diff.values))],percentile) 
            mkoffset = 1.01
            crossings = np.where(np.diff(r_std_mean_diff>thres_diff,1))[0]
            scatter = [ dfmpl.Close.iloc[i]*mkoffset if i in crossings else np.nan for i in xvals]
            crossup=np.where(np.diff((r_std_mean_diff>thres_diff)*1.,1)>0)[0]
            scatterup = [ dfmpl.Close.iloc[i]*mkoffset if i in crossup else np.nan for i in xvals]
            crossdn=np.where(np.diff((r_std_mean_diff>thres_diff)*1.,1)<0)[0]
            scatterdn = [ dfmpl.Close.iloc[i]*mkoffset if i in crossdn else np.nan for i in xvals]
            vlines = [dfmpl.index[i] for i in xvals if i in crossings ]
            my_vlines=[i for i in xvals if i in crossings ]

            predict = [] # capture data
            result= [] 
            entry_exit_pairs=(np.where(~np.isnan(scatterup))[0],np.where(~np.isnan(scatterdn))[0])
            for entry,exit in zip(*entry_exit_pairs):
                if entry-2<0:
                    continue
                s_1 = dfmpl.iloc[entry-2]
                s = dfmpl.iloc[entry-1]
                s1 = dfmpl.iloc[entry]
                s2 = dfmpl.iloc[exit]
                predict.append( (s_1.Close-s_1.Open,s.Close-s.Open) )
                result.append( s2.Close-s.Close )
            pred_sign = np.sign(predict,out=np.ones_like(np.asarray(predict))*-1.,where=np.asarray(predict)!=0)
            true_sign = np.sign(result,out=np.ones_like(np.asarray(result))*-1.,where=np.asarray(result)!=0)

            input_labels = [ "".join([ "1"if x>0 else "0" for x in [*i,z]]) for i,z in zip(pred_sign,true_sign) ] 
            predict_seq = sorted(Counter(input_labels).items()) 
            def mapped(i):
                if i[:2]=="00":
                    return -1
                if i[:2]=="11":
                    return 1
                return 0
            predicted_result = np.asarray([ mapped(i) for i in input_labels])
            lastNsamples = int(len(result)*0.1)

            profit=[] # implement backtesting for this strat
            trade_durations = []
            concated_df0=[]
            high_low_pair = []
            for entry,exit,buy in zip(*entry_exit_pairs,predicted_result): 
                s_1 = dfmpl.iloc[entry-2]
                s = dfmpl.iloc[entry-1]
                s1 = dfmpl.iloc[entry]
                s2 = dfmpl.iloc[exit]
                if buy==0:continue 
                if buy==-1:continue
                gains= buy*(s2.Close-s.Close)/(s.Close)
                profit.append(gains)
                # collecting trade stats
                indd=dfmpl.iloc[[entry,exit]].index
                diff = indd[1].to_datetime64()-indd[0].to_datetime64()
                trade_hr = int(diff)*1e-9/3600
                trade_durations.append( trade_hr )
                concated_df0.append( dfmpl.iloc[entry-1:exit+1] )
                high_low_pair.append( (s.Close,s2.Close,buy,
                                       dfmpl.iloc[entry:exit+1].High.max(),
                                       dfmpl.iloc[entry:exit+1].Low.min()) )
                #print(dfmpl.iloc[entry-1:exit+1])
                #print("*"*8,s.Close,s2.Close,buy,gains,entry,exit)
            def get_stats(lastNsamples=0):
                final_profit = 1
                profit1=profit[-lastNsamples:]
                ind=dfmpl.iloc[np.where(~np.isnan(scatterup))[0][np.r_[-lastNsamples,-1]]].index
                equity=[1]
                for p in profit1:
                    final_profit *= (1+p)
                    equity.append(final_profit)
                wins=sum(p>0 for p in profit1)
                diff = ind[1].to_datetime64()-ind[0].to_datetime64()
                trading_days = int(diff)*1e-9/3600/24
                gains_=final_profit*100-100
                str1=f"gain% = {gains_:.2f}%"+f", avg%= {np.mean(profit1)*100:.2f}%"+"\n"+\
                      f"trades={len(profit1)}, wins={wins}, win%={wins/len(profit1)*100:.2f}%"+"\n"+\
                      f"days={trading_days:.3g}d, trade dur={np.mean(trade_durations[-lastNsamples:]):.3g}hr"+"\n"+\
                f"gain%/day={gains_/trading_days:.2f}% per day"
                return str1,gains_,trading_days
            final_profit = 1
            profit1=profit
            equity=[1]
            for p in profit1:
                final_profit *= (1+p)
                equity.append(final_profit)

            fig,(ax1,ax2)=plt.subplots(2,1,figsize=(5,5),sharex=True)
            ax1.plot(equity,"-x")
            ax1.set_ylabel("equity (start with $1)")
            ax1.set_title(f"{tickerpair} {interval} {percentile}%")
            str1,gains__,trading_days = get_stats(0)
            ax1.text(0.02,0.99, "overall\n"+str1,transform=ax1.transAxes,va="top",ha="left")
            ax1.text(0.02,0.56, f"last {lastNsamples} trades\n"+get_stats(lastNsamples)[0],transform=ax1.transAxes,va="top",ha="left")
            ax1.text(0.87,0.0, "seq,n\n"+"\n".join([p[0]+","+str(p[1]) for p in predict_seq]),transform=ax1.transAxes,va="bottom",ha="left")
            ax1.axvline(len(profit)-lastNsamples,c="r",ls="--")
            ax2.plot(profit,"x")
            ax2.axhline(0,c="r",alpha=0.3)
            ax2.set_ylabel("profit per trade")
            ax2.text(0.02,0.99,f"full len={fulllen}",transform=ax2.transAxes,va="top",ha="left")
            for ax in (ax1,ax2):
                ax.set_xlabel("trades")
            plt.tight_layout()
            ax3 = ax2.inset_axes((0.5,0.02,0.3,0.3))

            ax3.hist(profit,bins=100)
            ax3.axvline(0,c="r",alpha=0.3)
            ax3.tick_params("x",top=True, labeltop=True, bottom=False, labelbottom=False,pad=-20,direction="in") 
            
            plt.tight_layout(h_pad=-1.08)
            if gains__>0:
                ax2.set_facecolor(f'#00{max(0,min(int(gains__)+100,255)):02X}0f60')
                plt.savefig(f"5_3figures_1_onlybuy/{int(gains__/trading_days*100):05d}_{tickerpair}_{interval}_{percentile}.png")
            else:
                ax2.set_facecolor(f'#{max(0,min(int(gains__)*-1+100,255)):02X}000f60')
                plt.savefig(f"5_3figures_1_onlybuy/{tickerpair}_{interval}_{percentile}_{int(gains__/trading_days)*-100:05d}_.png")
            plt.close("all")
            if doing%10==0:
                print(doing,end=" ")
            doing+=1

0 10 20 30 40 50 60 70 80 90 100 110 120 130 140 CPU times: total: 3min 50s
Wall time: 5min 33s


In [2]:
gains__

70.80167650903172